In [ ]:
pip install pulp

     |████████████████████████████████| 40.6MB 102kB/s 


In [ ]:
from pulp import *
import random

In [ ]:
modelo = LpProblem("Minimizar_Costo", LpMinimize) #definicion del modelo

In [ ]:
#Definición de conjuntos
Clientes = range(0,10) 
Depositos = range(0,2)
Mercados = range(1,6)
Centros_Acopio = range(1,5)

T_tras=[[random.randint(10,30) for col in range(1,11)] for row in range(1,11)]

#Funcion para obtener distancias
def get_dist(a,b):
  return ((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5   

#Coordenadas de los lugares
plantas=[(65,60),(2,70),(20,55),(30,10)]
mercados=[(5,65),(100,50),(25,15),(80,25),(95,85)]
acopios=[(60,65),(60,45),(70,85),(65,95)]

Planta_Mercado={}     

for i in Plantas:
  Planta_Mercado[i]={}
  for j in Mercados:
    Planta_Mercado[i][j]=(get_dist(plantas[i-1],mercados[j-1]))

Mercado_Acopio={}
for i in Mercados:
  Mercado_Acopio[i]={}
  for j in Centros_Acopio:
    Mercado_Acopio[i][j]=(get_dist(mercados[i-1],acopios[j-1]))

Planta_Acopio={}
for i in Plantas:
  Planta_Acopio[i]={}
  for j in Centros_Acopio:
    Planta_Acopio[i][j]=(get_dist(plantas[i-1],acopios[j-1]))
    
Acopio_desecho={}
for i in Centros_Acopio:
  Acopio_desecho[i]=(get_dist(acopios[i-1],(25,35)))



In [ ]:
#Definición de parámetros
#Costo de producción del producto j}
A = {1:15,2:15,3:15}
#Costo de transporte del producto j entre planta i y mercado k
B = {1:0.01455, 2:0.01455, 3:0.01455}
#Costo de transporte del producto j entre mercado k y centro de acopio l
C = {1:0.01455, 2:0.01455, 3:0.01455}
#Costo de transporte del producto j entre centro de acopio l y planta i
D = {1:0.003,2:0.003,3:0.003}
#Costo de transporte del producto j entre centro de acopio l y desechos
O = {1:0.00155,2:0.00155,3:0.00155}
#Costo fijo por aperturar planta i
E = {1:5000000,2:5000000,3:5000000,4:5000000}
#Costo fijo por aperturar centro de acopio l
F = {1:500000,2:500000,3:500000,4:500000}
#Costo ahorrado del producto j(producto j recuperado)
G = {1:7,2:7,3:7}
#Costo de enviar a desechos el producto j
H = {1:2.5,2:2.5,3:2.5}
#Capacidad de la planta i del producto j
P = {1:{1:84000,2:84000,3:84000},
     2:{1:84000,2:84000,3:84000},
     3:{1:84000,2:84000,3:84000},
     4:{1:84000,2:84000,3:84000}}
#Capacidad del centro de acopio l
Q = {1:{1:34000,2:34000,3:34000},
     2:{1:34000,2:34000,3:34000},
     3:{1:34000,2:34000,3:34000},
     4:{1:34000,2:34000,3:34000}}
#Demanda en el mercado K del producto j
d = {1:{1:30000,2:30000,3:30000},
     2:{1:30000,2:30000,3:30000},
     3:{1:30000,2:30000,3:30000},
     4:{1:30000,2:30000,3:30000},
     5:{1:30000,2:30000,3:30000},
     6:{1:30000,2:30000,3:30000}}
#Productos devueltos por el mercado k
r = {1:{1:10000,2:10000,3:10000},
     2:{1:10000,2:10000,3:10000},
     3:{1:10000,2:10000,3:10000},
     4:{1:10000,2:10000,3:10000},
     5:{1:10000,2:10000,3:10000},
     6:{1:10000,2:10000,3:10000}}
     
#Fracción de desecho del producto j
import random
alfa = {1:(random.uniform(0.27, 0.33)),2: (random.uniform(0.27, 0.33)), 3:(random.uniform(0.27, 0.33))}



In [ ]:
#Definición de variables

x = LpVariable.dicts("x",(Plantas,Mercados,Productos), lowBound= 0, cat = 'Integer') # Cantidad del producto j producido por la planta i para el mercado k
y = LpVariable.dicts("y",(Mercados,Centros_Acopio,Productos), lowBound = 0, cat = 'Integer') #Cantidad del producto j devuelto por el mercado k al centro de acopio l
s = LpVariable.dicts("s",(Centros_Acopio,Plantas,Productos), lowBound = 0, cat = 'Integer') #Cantidad del producto j devuelto por el centro de acopio l a la planta i
t = LpVariable.dicts("t",(Centros_Acopio,Productos), lowBound = 0, cat = 'Integer') #Cantidad del producto j enviado por el centro de acopio l a desechos
z = LpVariable.dicts("z",(Plantas), cat = 'Binary') #Si se ubica la planta 1 sino 0
w = LpVariable.dicts("w", (Centros_Acopio), cat = 'Binary') #Si se ubica el centro de acopio 1 sino 0



In [ ]:
  #Función Objetivo

modelo += lpSum(E[i]*z[i] for i in Plantas) + lpSum(F[l]*w[l] for l in Centros_Acopio)+lpSum(lpSum(lpSum((A[j]+B[j]*Planta_Mercado[i][k])*x[i][k][j] for j in Productos) for k in Mercados) for i in Plantas)+lpSum(lpSum(lpSum((C[j]*Mercado_Acopio[k][l])*y[k][l][j] for j in Productos) for l in Centros_Acopio) for k in Mercados) + lpSum(lpSum(lpSum((-G[j]+D[j]*Planta_Acopio[i][l])*s[l][i][j] for j in Productos) for i in Plantas) for l in Centros_Acopio) + lpSum(lpSum((H[j] + O[j]*Acopio_desecho[l])*t[l][j] for j in Productos) for l in Centros_Acopio)


In [ ]:
#Restricciones

#Asegurar la satisfacción de la demanda
for k in Mercados:
  for j in Productos:
    modelo += lpSum(x[i][k][j] for i in Plantas) >= d[k][j]
#Restricción de la capacidad de Planta
for i in Plantas:
  modelo += lpSum(lpSum(s[l][i][j] for j in Productos)for l in Centros_Acopio) + lpSum(lpSum(x[i][k][j] for j in Productos)for k in Mercados) <= z[i]*lpSum(P[i][j] for j in Productos)
#Restricción de fracción de productos devueltos
for k in Mercados:
  for j in Productos:
    modelo += lpSum(y[k][l][j] for l in Centros_Acopio) <= lpSum(x[i][k][j] for i in Plantas)
#Restricción de mínima fracción de producto devuelto
for l in Centros_Acopio:
  for j in Productos:
    modelo += alfa[j]*lpSum(y[k][l][j] for k in Mercados) <= t[l][j]
#Restricción de capacidad del centro de acopio
for l in Centros_Acopio:
  modelo += lpSum(lpSum(y[k][l][j] for j in Productos) for k in Mercados)<= w[l]*lpSum(Q[l][j] for j in Productos)
#Restricción del total de producto devuelto
for l in Centros_Acopio:
  for j in Productos:
    modelo += lpSum(y[k][l][j] for k in Mercados) == lpSum(s[l][i][j] for i in Plantas) + t[l][j]
#Restricción de cantidad de productos devueltos
for k in Mercados:
  for j in Productos:
    modelo += lpSum(y[k][l][j] for l in Centros_Acopio) == r[k][j]



In [ ]:
modelo.solve()


1

In [ ]:
for v in modelo.variables():
  if v.varValue > 0:
    print(v.name, "=", v.varValue)

s_2_3_1 = 20123.0
s_2_3_2 = 22295.0
s_2_3_3 = 11582.0
t_2_1 = 9877.0
t_2_2 = 9705.0
t_2_3 = 28418.0
t_3_1 = 20000.0
t_3_2 = 18000.0
t_3_3 = 10000.0
w_2 = 1.0
w_3 = 1.0
x_1_2_1 = 30000.0
x_1_2_2 = 30000.0
x_1_2_3 = 30000.0
x_1_4_1 = 30000.0
x_1_4_2 = 30000.0
x_1_4_3 = 12000.0
x_1_5_1 = 30000.0
x_1_5_2 = 30000.0
x_1_5_3 = 30000.0
x_3_1_1 = 30000.0
x_3_1_2 = 30000.0
x_3_1_3 = 30000.0
x_3_3_1 = 30000.0
x_3_3_2 = 30000.0
x_3_3_3 = 30000.0
x_3_4_3 = 18000.0
y_1_2_1 = 10000.0
y_1_2_2 = 10000.0
y_1_2_3 = 10000.0
y_2_2_2 = 2000.0
y_2_2_3 = 10000.0
y_2_3_1 = 10000.0
y_2_3_2 = 8000.0
y_3_2_1 = 10000.0
y_3_2_2 = 10000.0
y_3_2_3 = 10000.0
y_4_2_1 = 10000.0
y_4_2_2 = 10000.0
y_4_2_3 = 10000.0
y_5_3_1 = 10000.0
y_5_3_2 = 10000.0
y_5_3_3 = 10000.0
z_1 = 1.0
z_3 = 1.0
